# Simple vector interpolation examples

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import sys, os
sys.path.append(os.path.relpath("..//..//..//src//python"))
import hiped as hp

## 1) Domain definition

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

D = [0]
def f0(x):
    dico = {"Pentagon" : 5, "Diamond" : "diamond3"}
    D[0] = hp.Domain(dico[x])
    plt.clf()
    D[0].plot()
    plt.show()

plt.figure()
_ = interact(lambda x : f0(x), x= ["Pentagon", "Diamond"])

## 2) Definition of vector functions at each vertex

In [ ]:
from hiped.utils import mult

dimInput = 2
dimOutput = 2

f1v = hp.VertexFunction(label = "f1v", f = lambda u : u**2, dfdu = lambda u : 2*u * np.eye(2)[:,:,None], dimInput= dimInput, dimOutput = dimOutput)
f2v = hp.VertexFunction("f2v", lambda u : u**3, lambda u : 3*u**2 * np.eye(2)[:,:,None], dimInput, dimOutput)
f3v = hp.VertexFunction("f3v", lambda u : 0.1*u, lambda u : 0.1 * np.eye(2)[:,:,None] * np.ones(u.shape), dimInput, dimOutput)
f4v = hp.VertexFunction("f4v", lambda u : np.ones(u.shape), lambda u : np.zeros(u.shape) * np.zeros((2,2,1)), dimInput, dimOutput)
f5v = hp.VertexFunction("f5v", lambda u : mult(np.array([[1,2],[3,4]]), u), lambda u : np.array([[1,2],[3,4]])[:,:,None] * np.ones(u.shape), dimInput, dimOutput)

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f1(x):
    plt.clf()
    x.plot()
    plt.show()

plt.figure()
_ = interact(lambda x : f1(x), x= [f1v, f2v, f3v, f4v, f5v]) 
# ugly because of matplotlib 3D rendering...

## 3) Definition of the interpolation
We can chose a penalization $P$ applied to the shape functions $\omega_i$, which depend on the ```Domain``` (it is also possible to apply different penalizations $P_i$ for each shape function): 
$$ f(x,u) = \sum_{i=1}^{n_m} P(\omega_i(x)) f_i(u) $$

In [ ]:
P = [1]
def f2(x,y,z, P):
    plt.clf()
    P[0] = hp.Penalization(x, y, z)
    P[0].plot()
    plt.show()

plt.figure()
p = widgets.FloatSlider(min=0., max=5., step=0.1, value=2.)
type = ["SIMP", "RAMP", "Lukas", "Zhou"]
reversed = widgets.Checkbox(value=False)
interact(lambda type,p, reversed : f2(type, p, reversed, P), type=type, p = p, reversed =reversed )

interpVector= hp.Interpolation(domain = D[0], children = [f1v,f2v,f3v,f4v,f5v],
                               label = "interpVector", penalization = P[0])

## 4) Coordinates in the polygon $x$ and scalar field $u$
Then the $n$ cartesian coordinates $x$ and the scalar field $u$ should be defined to evaluate the interpolation.

In [ ]:
x, u = [0], [0]

def f3(n, typeInit, r, u, x):
    plt.clf()
    x[0] = interpVector.setInitialVariable(n, typeInit = typeInit, radius = r) # initialization of the variables
    x[0] = interpVector.projection(x[0]) # projection onto the domain
    interpVector.plot(x[0])
    u[0] = np.random.rand(dimInput,1,n)
    plt.show()

n = widgets.IntSlider(min=1, max=1000, step=1, value=100)
typeInit = ["rand","zero"]
r = widgets.FloatSlider(min=0, max=2, step=0.1, value=1)
plt.figure()
_ = interact(lambda n, typeInit, r : f3(n, typeInit, r, u, x), n=n, typeInit = typeInit, r =r )    

## 5) Evaluation of the interpolation
### a) Interpolated values

In [ ]:
x, u = x[0], u[0]
import time
nRuns = 1000
t0 = time.time()
for i in range(nRuns):
    f = interpVector.eval(x, u) # evaluate the interpolation
tv1 = time.time() - t0
print(f"Compute interpolated values in {tv1*1000:.3f} ms ({u.shape[2]} values, {nRuns} runs)")

### b) Derivative with respect to the vector field $u$

In [ ]:
nRuns = 1000
t0 = time.time()
for i in range(nRuns):
    dfdu =  interpVector.evaldu(x, u) # evaluate the derivative of the interpolation w.r.t u
tv1 = time.time() - t0
print(f"Compute u-derivative in {tv1*1000:.3f} ms ({u.shape[2]} values, {nRuns} runs)")

#### Check the Taylor expansion

We introduce small perturbations and check if the finite difference estimation of the derivative converges to the computed one.

In [ ]:
from hiped.utils import mult

h = np.logspace(-8,-2,10) # test for 10 different h
resU = np.zeros((10,1,u.shape[2]))


for i in range(10):
    pert = h[i]*np.random.rand(*u.shape);
    fPerturbedu = interpVector.eval(x,u+pert);
    resU[i,0,:] = np.abs(np.linalg.norm(fPerturbedu - (f + mult(dfdu,pert)), axis = 0))
    
maxResU = np.max(resU, axis = 2); medResU = np.median(resU, axis = 2);

plt.figure()
plt.loglog(h, medResU,'-o', label =  "median of the residual")
plt.loglog(h, maxResU,'-o', label =  "maximum of the residual")
plt.loglog(h, h**2,'k--', label =  "expected decay $(h^2)$")

plt.legend(loc = "best"); plt.grid()
plt.xlabel("$h$"); plt.ylabel("Euclidian norm of Taylor remainder")
plt.title("Taylor remainder with respect to vector field $u$")

plt.show()

### c) Derivative with respect to the cartesian coordinates in the polygon $x$

In [ ]:
nRuns = 500
t0 = time.time()
for i in range(nRuns):
    dfdx =  interpVector.evaldx(x, u) # evaluate the derivative of the interpolation w.r.t x
tv1 = time.time() - t0
print(f"Compute u-derivative in {tv1*1000:.3f} ms ({u.shape[2]} values, {nRuns} runs)")

#### Check the Taylor expansion

We introduce small perturbations and check if the finite difference estimation of the derivative converges to the computed one.

In [ ]:
resX = np.zeros((10,1,u.shape[2]))
l = list(x.keys())[0]

for i in range(10):
    xPert = x.copy()
    pert = h[i]*np.random.rand(*x[l].shape)
    xPert[l] = x[l] + pert
    fPerturbedx = interpVector.eval(xPert,u)
    pert = np.reshape(pert.T, (2,1,-1))
    resX[i,0,:] = np.linalg.norm(fPerturbedx - (f + mult(dfdx[l],pert)), axis = 0)

maxResX = np.max(resX, axis = 2); medResX = np.median(resX, axis = 2);

plt.figure()
plt.loglog(h, medResX,'-o', label =  "median of the residual")
plt.loglog(h, maxResX,'-o', label =  "maximum of the residual")
plt.loglog(h, h**2,'k--', label =  "expected decay $(h^2)$")

plt.legend(loc = "best"); plt.grid()
plt.xlabel("$h$"); plt.ylabel("Euclidian norm of Taylor remainder")
plt.title("Taylor remainder with respect to $x$")

plt.show()

### d) Speed tip

To compute several times the interpolation for different $u$  without changing the position $x$ in the domain (for example, when solving a non-linear system in $u$), one can first compute the shape functions once for all.

In [ ]:
u2 = np.random.rand(dimInput,nRuns,u.shape[2])

# naive
t0 = time.time()
for i in range(nRuns):
    interpVector.eval(x, u2[:,i,:]) # value
    interpVector.evaldu(x, u2[:,i,:]) # derivative w.r.t u
    interpVector.evaldx(x, u2[:,i,:]) # derivative w.r.t x

tNaive = time.time() - t0
print(f"Computation time (naive version, {nRuns} runs) : {tNaive*1000:.3f} ms")

# faster
t0 = time.time()
# pre-computation of the shape functions that don't depend on u
w, dwdx = interpVector.evalBasisFunction(x)
for i in range(nRuns):
    interpVector.eval(x, u2[:,i,:], w) # value
    interpVector.evaldu(x, u2[:,i,:], w) # derivative w.r.t u
    interpVector.evaldx(x, u2[:,i,:], w, dwdx) # derivative w.r.t x
    
tOptim = time.time() - t0
print(f"Computation time (optimized version, {nRuns} runs) : {tOptim*1000:.3f} ms")